# Facebook Posts - Source API
Get the source data for facebook posts from crowdtangle api.

Requires environment variables:
- CROWDTANGLE_API_TOKEN
- CT_RATE_LIMIT_CALLS

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

import pandas as pd

from phoenix.common import artifacts
from phoenix.common import utils
from phoenix.common import run_datetime
from phoenix.scrape import scrape
from phoenix.scrape import crowdtangle

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parameters
RUN_DATETIME = None

# Set Artefacts URLs
# ARTIFACTS_ENVIRONMENT_KEY will only effect the URLs that are not set
ARTIFACTS_ENVIRONMENT_KEY = "local"
ARTIFACT_SOURCE_FB_POSTS_URL = None

# The period you want to scrape
# Default three days before run date
SCRAPE_START_DATE = None
SCRAPE_END_DATE = None
SCRAPE_LIST_IDS = ["1531680"]

In [ ]:
# DEAFULTS
if RUN_DATETIME:
    run_dt = run_datetime.from_file_safe_str(RUN_DATETIME)
else:
    run_dt = run_datetime.create_run_datetime_now()
    
aur = artifacts.registry.ArtifactURLRegistry(run_dt, ARTIFACTS_ENVIRONMENT_KEY)
if ARTIFACT_SOURCE_FB_POSTS_URL is None:
    ARTIFACT_SOURCE_FB_POSTS_URL = aur.get_url("source-posts")

if SCRAPE_START_DATE is None:
    SCRAPE_START_DATE = run_dt.dt - datetime.timedelta(days=3)
    
if SCRAPE_END_DATE is None:
    SCRAPE_END_DATE = run_dt.dt

In [ ]:
# Display params.
print(
ARTIFACT_SOURCE_FB_POSTS_URL,
run_dt.dt,
SCRAPE_START_DATE,
SCRAPE_END_DATE,
SCRAPE_LIST_IDS,
sep='\n',
)

In [ ]:
# Scrape code will go here
posts = crowdtangle.get_all_posts(SCRAPE_START_DATE, SCRAPE_END_DATE, SCRAPE_LIST_IDS)
len(posts)

In [ ]:
posts[0:1]

In [ ]:
a = artifacts.json.persist(ARTIFACT_SOURCE_FB_POSTS_URL, posts)
a.url

In [ ]:
a = artifacts.json.persist(ARTIFACT_BASE_TO_PROCESS_FB_POSTS_URL, posts)
a.url